In [1]:
install.packages("readxl")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library("readxl")
df<-read_excel('final.xlsx')

In [3]:
df<-df[2:17]
print(df)

# A tibble: 74 × 16
   `Mean Velocity` `Standard Deviation` `Max Velocity` `Min Velocity`
             <dbl>                <dbl>          <dbl>          <dbl>
 1       -8.44e-13             3.53e-10  0.00000000787       -8.19e-9
 2       -1.94e-12             3.87e-10  0.00000000471       -4.60e-9
 3       -2.98e-13             3.22e-10  0.00000000597       -6.14e-9
 4       -1.55e-13             3.38e-10  0.00000000685       -6.16e-9
 5       -6.92e-13             3.01e-10  0.00000000549       -4.48e-9
 6       -7.79e-13             5.56e-10  0.0000000193        -1.91e-8
 7        1.24e-13             1.60e- 9  0.0000000481        -5.28e-8
 8       -5.62e-13             7.32e-10  0.0000000196        -2.02e-8
 9       -5.62e-13             7.32e-10  0.0000000196        -2.02e-8
10       -4.88e-13             2.77e-10  0.00000000450       -4.71e-9
# ℹ 64 more rows
# ℹ 12 more variables: `Range Velocity` <dbl>, `Median Velocity` <dbl>,
#   `RMS Velocity` <dbl>, Energy <dbl>, `Impulse Fa

In [4]:
df[[16]] <- as.factor(df[[16]])
print(df[16])
str(df[16])

# A tibble: 74 × 1
   mq_type  
   <fct>    
 1 impact_mq
 2 impact_mq
 3 impact_mq
 4 impact_mq
 5 deep_mq  
 6 impact_mq
 7 impact_mq
 8 impact_mq
 9 deep_mq  
10 impact_mq
# ℹ 64 more rows
tibble [74 × 1] (S3: tbl_df/tbl/data.frame)
 $ mq_type: Factor w/ 3 levels "deep_mq","impact_mq",..: 2 2 2 2 1 2 2 2 1 2 ...


In [5]:
install.packages('caTools')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [6]:
library(caTools)
set.seed(123)
split<-sample.split(df$mq_type, SplitRatio = 0.9)
training_set<-subset(df, split==TRUE)
test_set<-subset(df, split==FALSE)

In [7]:
numeric_cols <- sapply(training_set, is.numeric)
training_set[numeric_cols] <- scale(training_set[numeric_cols])
train_means <- attr(scale(training_set[numeric_cols]), "scaled:center")
train_sds <- attr(scale(training_set[numeric_cols]), "scaled:scale")
test_set[numeric_cols] <- scale(test_set[numeric_cols], center = train_means, scale = train_sds)

In [8]:
install.packages('h2o')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [9]:
library(h2o)


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit https://docs.h2o.ai

----------------------------------------------------------------------



Attaching package: ‘h2o’


The following objects are masked from ‘package:stats’:

    cor, sd, var


The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc




In [10]:
h2o.init(nthreads = -1)

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         32 minutes 47 seconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.44.0.3 
    H2O cluster version age:    1 year, 3 months and 29 days 
    H2O cluster name:           H2O_started_from_R_root_ozp357 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   3.17 GB 
    H2O cluster total cores:    2 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:                  R version 4.5.0 (2025-04-11) 


Warning message in h2o.clusterInfo():
“
Your H2O cluster version is (1 year, 3 months and 29 days) old. There may be a newer version available.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html”


In [11]:
classifier = h2o.deeplearning(y= 'mq_type',
training_frame = as.h2o(training_set), activation = 'Rectifier',
hidden = c(10,10), # Taking 10 since it's avg of i/p and o/p
epoch = 100, train_samples_per_iteration = -2 ) # h2o model auto tunes

  |======================================================================| 100%


Warning message in .h2o.processResponseWarnings(res):
“Dropping bad and constant columns: [Arrival Time].
”


  |======================================================================| 100%


In [12]:
#prob_predict<-h2o.predict(classifier, newdata = as.h2o(test_set[-1]))
prob_predict <- h2o.predict(classifier, newdata = as.h2o(test_set))

  |======================================================================| 100%
  |======================================================================| 100%


In [13]:
prob_predict <- h2o.predict(classifier, newdata = as.h2o(test_set))

  |======================================================================| 100%
  |======================================================================| 100%


In [14]:
#y_pred<-as.vector(y_pred)
y_pred <- as.vector(prob_predict$predict)

In [15]:
#y_pred<-(prob_predict>0.5)

In [16]:
print(y_pred)

[1] "impact_mq" "impact_mq" "impact_mq" "impact_mq" "impact_mq" "impact_mq"
[7] "impact_mq"


In [17]:
cm <- table(test_set$mq_type, y_pred)
print(cm)

            y_pred
             impact_mq
  deep_mq            1
  impact_mq          6
  shallow_mq         0


In [18]:
correct_predictions <- sum(diag(cm))
total_predictions <- sum(cm)
accuracy <- correct_predictions / total_predictions

print(paste("Accuracy:", round(accuracy * 100, 2), "%"))

[1] "Accuracy: 14.29 %"


In [19]:
classifier = h2o.deeplearning(y= 'mq_type',
training_frame = as.h2o(training_set), activation = 'Tanh',
hidden = c(10,10), # Taking 10 since it's avg of i/p and o/p
epoch = 100, train_samples_per_iteration = -2 ) # h2o model auto tunes

  |======================================================================| 100%


Warning message in .h2o.processResponseWarnings(res):
“Dropping bad and constant columns: [Arrival Time].
”


  |======================================================================| 100%


In [20]:
prob_predict <- h2o.predict(classifier, newdata = as.h2o(test_set))

  |======================================================================| 100%
  |======================================================================| 100%


In [21]:
prob_predict <- h2o.predict(classifier, newdata = as.h2o(test_set))

  |======================================================================| 100%
  |======================================================================| 100%


In [22]:
y_pred <- as.vector(prob_predict$predict)

In [23]:
print(y_pred)

[1] "impact_mq" "impact_mq" "impact_mq" "impact_mq" "impact_mq" "impact_mq"
[7] "impact_mq"


In [24]:
cm <- table(test_set$mq_type, y_pred)
print(cm)

            y_pred
             impact_mq
  deep_mq            1
  impact_mq          6
  shallow_mq         0


In [25]:
correct_predictions <- sum(diag(cm))
total_predictions <- sum(cm)
accuracy <- correct_predictions / total_predictions

print(paste("Accuracy:", round(accuracy * 100, 2), "%"))

[1] "Accuracy: 14.29 %"
